In [247]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [248]:
gemini_api_key = os.getenv("GEMINI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")

os.environ["TAVILY_API_KEY"] = tavily_api_key
os.environ["SERPER_API_KEY"] = serper_api_key

In [249]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=gemini_api_key)

In [250]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=gemini_api_key)

In [251]:
llm.invoke("hi,how are you?").content

'I am doing well, thank you for asking! How are you today?'

predefine tools

In [252]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=api_wrapper)
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Retrieval-augmented generation
Summary: Retrieval-augmented generation (RAG) is a technique that enables generative artificial intelligence (Gen AI) models to retrieve and incorporate new information. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to supplement information from its pre-existing training data. This allows LLMs to use domain-specific and/or updated information. Use cases include providing chatbot access to internal company data or generating responses based on authoritative sources.
RAG improves larg

In [253]:
from langchain_community.tools import YouTubeSearchTool
tool2 = YouTubeSearchTool()
tool2.run("carry minati")

"['https://www.youtube.com/watch?v=m9s1NQG3TNY&pp=ygUMY2FycnkgbWluYXRp', 'https://www.youtube.com/watch?v=GOFQN8otiYs&pp=ygUMY2FycnkgbWluYXRp']"

In [254]:
from langchain_community.tools.tavily_search import TavilySearchResults
tool3 = TavilySearchResults()
tool3.invoke({"query": "who is mohandas karamchand gandhi?"})

[{'title': 'Mohandas Karamchand Gandhi | South African History Online',
  'url': 'https://www.sahistory.org.za/people/mohandas-karamchand-gandhi',
  'content': 'Mohandas Karamchand Gandhi\nSynopsis\nlawyer, founder of the Natal Indian Congress, political prisoner, leader of the Indian community in South Africa, political activist against the White South African Government and the British Government in India\nFirst name\nMohandas\nMiddle name\nKaramchand\nLast name\nGandhi\nDate of birth\n02-October-1869\nLocation of birth\nPorbandar, Gujarat, India\nDate of death\n30-January-1948\nLocation of death\nNew Delhi, India\nGender\nMale [...] Mohandas Karamchand Gandhi was born to a Hindu family on 2 October 1869, in Porbandar, Gujarat, India. He was the last child of Karamchand Gandhi, his father and his father\'s fourth wife Putlibai. His father,\xa0 a lawyer and an\xa0important government official, belonged to the merchant caste. His early schooling was in nearby Rajkot, where his father s

Create a Custom tool

In [255]:
from langchain.agents import tool
@tool
def get_word_length(word: str) -> int:
    """Return the length of a word."""
    return len(word)

In [256]:
get_word_length("abnc")

4

In [257]:
@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers."""
    return a*b

In [258]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [259]:
multiply.invoke({"a":10,"b":20})

200

Concept of Agents
This Agent class from the first version of LangChain(lagacy)
they updated it towards the different classes
https://python.langchain.com/docs/concepts/

In [260]:
from langchain.agents import AgentType, load_tools, initialize_agent

tool = load_tools(["wikipedia"],llm=llm)

In [261]:
agent = initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [262]:
agent.run("what is llama and who create this llm model?")



> Entering new AgentExecutor chain...
I need to find out what LLaMA is and who created it. I can use Wikipedia to find this information.
Action: wikipedia
Action Input: LLaMA LLM
Observation: Page: Llama (language model)
Summary: Llama (Large Language Model Meta AI, formerly stylized as LLaMA) is a family of large language models (LLMs) released by Meta AI starting in February 2023. The latest version is Llama 4, released in April 2025.
Llama models are trained at different parameter sizes, ranging between 1B and 405B. Initially  only a foundation model, starting with Llama 2, Meta AI released instruction fine-tuned versions alongside foundation models.
Model weights for the first version of Llama were only available to researchers on a case-by-case basis, under a non-commercial license. Unauthorized copies of the first model were shared via BitTorrent. Subsequent versions of Llama were made accessible outside academia and released under licenses that permitted some commercial use.
A

'LLaMA is a family of large language models released by Meta AI.'

A Tool Calling Agent refers to an AI system or framework that dynamically invokes external tools or APIs to accomplish tasks as part of its decision-making process. Instead of relying solely on internal reasoning or knowledge, the agent calls specific tools to perform actions like calculations, searches, or other operations. This type of agent is often used in real-world applications where tasks require interaction with external systems.

In [263]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()
search.invoke("what is the weather in SF")

[{'title': 'Weather in San Francisco',
  'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1744099881, 'localtime': '2025-04-08 01:11'}, 'current': {'last_updated_epoch': 1744099200, 'last_updated': '2025-04-08 01:00', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 5.8, 'wind_kph': 9.4, 'wind_degree': 278, 'wind_dir': 'W', 'pressure_mb': 1022.0, 'pressure_in': 30.19, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 86, 'cloud': 25, 'feelslike_c': 12.1, 'feelslike_f': 53.7, 'windchill_c': 11.1, 'windchill_f': 51.9, 'heatindex_c': 12.0, 'heatindex_f': 53.6, 'dewpoint_c': 11.4, 'dewpoint_f': 52.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 8.8, 'gust_kph': 14.2}}",
  's

In [264]:
tools = [search]

In [265]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [266]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm,tools,prompt)

In [267]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

agent_executor.invoke({"input":"hello how are you?"})



> Entering new AgentExecutor chain...
I am doing well, thank you for asking. How can I help you today?

> Finished chain.


{'input': 'hello how are you?',
 'output': 'I am doing well, thank you for asking. How can I help you today?'}

In [268]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'title': 'San Francisco weather in April 2025 | Weather25.com', 'url': 'https://www.weather25.com/north-america/usa/california/san-francisco?page=month&month=April', 'content': '| Sun | Mon | Tue | Wed | Thu | Fri | Sat |\n| --- | --- | --- | --- | --- | --- | --- |\n| \n | \n| 1  53° /48° | 2  53° /46° | 3  53° /46° | 4  62° /48° | 5  64° /53° |\n| 6  57° /51° | 7  59° /50° | 8  60° /50° | 9  64° /53° | 10  64° /55° | 11  59° /53° | 12  55° /48° |\n| 13  59° /46° | 14  59° /48° | 15  60° /48° | 16  62° /48° | 17  60° /48° | 18  60° /50° | 19  60° /50° |\n| 20  60° /48° | 21  62° /50° | 22  62° /50° | 23  62° /50° | 24  62° /50° | 25  60° /50° | 26  64° /50° | [...] | July | 77° / 57° | 0 | 31 | 0 | 0.12 in | \nPerfect\n| San Francisco in July |\n| August | 77° / 57° | 0 | 31 | 0 | 0.12 in | \nPerfect\n| San Francisco in August |\n| September | 77° / 57° | 0 

{'input': 'whats the weather in sf?',
 'output': 'The weather in San Francisco is partly cloudy with a temperature of 55°F. The wind is blowing from the west at 5.8 mph.'}

1. RAG tool
2. ReAct
3. custom tool with ReAct agent
4. agent code from latest versions(v0.2 and v0.3)

In [269]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/")
docs = loader.load()

In [270]:
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

In [271]:
vector = FAISS.from_documents(documents,embeddings)
retriever = vector.as_retriever()

In [272]:
retriever.invoke("what is langgrapg studio? ")[0]

Document(id='4e6aa952-577d-4fff-b8b7-86b0c5b5d503', metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content="is needed to make the development process easier. Building LLM applications differs from traditional software development, requiring different tooling outside of the traditional code editor.Coding is still important to developing LLM applications — after all, production-ready LangGraph applications have complicated custom logic in the nodes and edges of the graphs that are created. We don't aim to replace code editors but, instead, to augment the development experience with tools tailored for LangGraph applications. LangGraph Studio facilitates this by making it easy to visualize an

In [273]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langGraph_search",
    "Search for information about langGraph. For any questions about langGraph, you must use this tool!",
)

In [274]:
tools = [search, retriever_tool]

In [275]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm,tools, prompt)

In [276]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

agent_executor.invoke({"input":"hi! what is langgraph studio and how to use it?"})



> Entering new AgentExecutor chain...

Invoking: `langGraph_search` with `{'query': 'langgraph studio and how to use it'}`


a specific node) halfway through the agent's trajectory. This creates an iterative process, by letting you interact with and manipulate the state at that point in time.While there is much more to explore, we're excited to introduce LangGraph Studio to start with bringing some of the core features of an agent IDE to the world.How to use LangGraph StudioLangGraph Studio is a desktop app, currently available for Apple Silicon. You can download a version here. Support for more platforms is coming soon.After you download and open LangGraph Studio, you will be prompted to log in with your LangSmith account. All users of LangSmith (including those with free accounts) currently have access to LangGraph Studio while it is in beta. You can sign up for a LangSmith account here.After downloading LangSmith, you can open a directory. At a bare minimum, this directory needs t

{'input': 'hi! what is langgraph studio and how to use it?',
 'output': "LangGraph Studio is the first IDE designed specifically for agent development. It's a desktop app that allows you to visualize, interact with, and debug complex agentic applications. It helps developers understand the structure of their graphs and modify agent results or logic.\n\nTo use it:\n\n1.  Download the app (currently available for Apple Silicon).2.  Log in with your LangSmith account (free accounts have access during the beta).\n3.  Open a directory containing a Python file with a graph defined in it and a `langgraph.json` file containing details.\n\nIt seems to augment the development experience by providing tools tailored for LangGraph applications."}

In [277]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...
I'm sorry, I don't have the ability to get weather information. I can only use the tools available to me, which are a search engine and a tool for looking up information about LangGraph.

> Finished chain.


{'input': 'whats the weather in sf?',
 'output': "I'm sorry, I don't have the ability to get weather information. I can only use the tools available to me, which are a search engine and a tool for looking up information about LangGraph."}

In [278]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

In [279]:
google_search = GoogleSerperAPIWrapper(serper_api_key=serper_api_key)
tools = [
    Tool(
        name="Intermediate Answer",
        func=google_search.run,
        description="useful for when you need to ask with search",
        verbose = True
    )
]

In [280]:
template = '''Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}'''

In [281]:
prompt = PromptTemplate.from_template(template)

search_agent = create_react_agent(llm,tools,prompt)

In [282]:
agent_executor = AgentExecutor(
    agent=search_agent,
    tools = tools,
    verbose= True,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)

In [283]:
agent_executor.invoke({"input": "Where is the hometown of the 2011 cricket worldcup captain winner ?"})



> Entering new AgentExecutor chain...
I need to find out who the captain of the 2011 Cricket World Cup-winning team was, and then find out where that person's hometown is.

Action: Intermediate Answer
Action Input: "who was the captain of the 2011 cricket world cup winning team"India national cricket teamI already know the 2011 Cricket World Cup-winning team was India. I now need to find out who the captain was.

Action: Intermediate Answer
Action Input: "who was india's captain in 2011 cricket world cup"Indian captain Mahendra Singh Dhoni had then said he preferred having three seamers because it gave him more options. He already had proven spinners in Harbhajan and Yuvraj, and could call upon many other Indian players who can bowl part-time spin (including Sachin Tendulkar, Suresh Raina, and Virender Sehwag).The captain of the 2011 Cricket World Cup-winning team (India) was Mahendra Singh Dhoni. Now I need to find his hometown.

Action: Intermediate Answer
Action Input: "Mahendra S

{'input': 'Where is the hometown of the 2011 cricket worldcup captain winner ?',
 'output': 'Ranchi, Bihar (now in Jharkhand)',
 'intermediate_steps': [(AgentAction(tool='Intermediate Answer', tool_input='who was the captain of the 2011 cricket world cup winning team', log='I need to find out who the captain of the 2011 Cricket World Cup-winning team was, and then find out where that person\'s hometown is.\n\nAction: Intermediate Answer\nAction Input: "who was the captain of the 2011 cricket world cup winning team"'),
   'India national cricket team'),
  (AgentAction(tool='Intermediate Answer', tool_input="who was india's captain in 2011 cricket world cup", log='I already know the 2011 Cricket World Cup-winning team was India. I now need to find out who the captain was.\n\nAction: Intermediate Answer\nAction Input: "who was india\'s captain in 2011 cricket world cup"'),
   'Indian captain Mahendra Singh Dhoni had then said he preferred having three seamers because it gave him more opti

ReAct Agent with Custom tools

In [284]:
from langchain.tools import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

In [285]:
@tool
def get_employee_id(name):
  """
  To get employee id, it takes employee name as arguments
  name(str): Name of the employee
  """
  fake_employees = {
    "Alice": "E001",
    "Bob": "E002",
    "Charlie": "E003",
    "Diana": "E004",
    "Evan": "E005",
    "Fiona": "E006",
    "George": "E007",
    "Hannah": "E008",
    "Ian": "E009",
    "Jasmine": "E010"}
  
  return fake_employees.get(name,"Employee not found")

In [286]:
@tool
def get_employee_salary(employee_id):
  """
  To get the salary of an employee, it takes employee_id as input and return salary
  """
  employee_salaries = {
    "E001": 56000,
    "E002": 47000,
    "E003": 52000,
    "E004": 61000,
    "E005": 45000,
    "E006": 58000,
    "E007": 49000,
    "E008": 53000,
    "E009": 50000,
    "E010": 55000
    }
  return employee_salaries.get(employee_id,"Employee not found")

In [287]:
prompt = hub.pull("hwchase17/react")

print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [288]:
tools1 = [get_employee_id, get_employee_salary]

agent = create_react_agent(llm,tools1,prompt)

agent_executor = AgentExecutor(agent=agent,tools=tools1,verbose=True)

In [289]:
agent_executor.invoke({"input":"What is the Salary of sunny?"})



> Entering new AgentExecutor chain...
I need to find the salary of sunny. First, I need to find sunny's employee ID.
Action: get_employee_id
Action Input: sunnyEmployee not foundI apologize, but I am unable to find the employee id for sunny.
Final Answer: Employee not found

> Finished chain.


{'input': 'What is the Salary of sunny?', 'output': 'Employee not found'}

In [290]:
agent_executor.invoke({"input":"What is the Salary of Evan?"})



> Entering new AgentExecutor chain...
I need to find the salary of Evan. First I need to find Evan's employee ID.
Action: get_employee_id
Action Input: EvanE005I now have Evan's employee ID, which is E005. Now I need to find the salary for employee E005.
Action: get_employee_salary
Action Input: E00545000I now know the salary of Evan.
Final Answer: 45000

> Finished chain.


{'input': 'What is the Salary of Evan?', 'output': '45000'}